In [1]:
!pip install pandas-gbq
!pip install yfinance
import pandas as pd
import yfinance as yf
import requests
import json 
import csv
import datetime
import os
import glob
import io
#from google.cloud import bigquery
#from google.oauth2 import service_account

/bin/bash: pip: command not found
/bin/bash: pip: command not found


/Users/nickwarner/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(




Manually load list of tickers

In [2]:
data = '''AMZN
INTC
LRCX
META
BABA
GIS
KMI
KO
MCD
MDT
MMM
TEL
USB
XYL
URNM
ABNB
AMAT
ASML
KLAC
KLIC
MRAM
SWKS
UROY
VMFXX
BWXT
DOCS
SPOT
SILXY
CLLNY
SRUUF
YLLXF
IGN
VGT
VUG
SOXX
VONG
JEPI
SCHD
VB
VIG
VTI
FXAIX
SPAXX
VXUS
TWCGX
SPY
EMCLOUD
COMP
FROG
MCHP
IRDM
NVEC
MXL
OPRA
BELFB
EXTR
IDCC
ON
HLIT
PLAB
VEEV
TTD
SHOP
CDLX
ZI
DV
S
ZS
CRWD
ADSK
PANW
MSI
ANET
EQIX
EGIO
DOCN
NET
RKLB
ASTS
COIN
NCNO
FISV
ENPH
TOELY
QRVO
BESI
VSH
AEHR
TXN
SLAB
CRUS
NXPI
AVGO
MPWR
LSCC
SNPS
CDNS
APH
CGNX
PLPC
SYM
IOT
TMO
DHR
TT
IRM
SEIC
NOW
BDC
DIOD
JBL
CARG
CSGS
ADBE
CRM
NOW
SHOP
WDAY
PYPL
SNOW
CRWD
TEAM
DDOG
SQ
MDB
VEEV
ZS
HUBS
NET
ZM
OKTA
TWLO
PAYC
PATH
DBX
PCTY
DOCU
TOST
ESTC
PCOR
APPF
GTLB
MNDY
QLYS
BILL
SMAR
CFLT
FRSH
WIX
ZI
WK
S
FIVN
BRZE
TENB
ASAN
SQSP
CXM
CWAN
BOX
BL
NCNO
AI
SPT
FROG
RNG
DOCN
FSLY
PD
QTWO
AVDX
AVPT
AMPL
ZUO
EVBG
DH
YEXT
OLO
BIGC
VMEO
WEAV
EGHT
DOMO
NOVT
IPGP
COHR
GFS
MU
WOLF
TXN
INTC
NVDA
ADI
QCOM
NXPI
TER
ALGM
MRVL
SWKS
ASML
AMD
MPWR
ON
LSCC
QRVO
AVGO
MCHP
AMAT
LRCX
ENTG
KLAC
TSM
SYNA
MKL'''

tickers = data.split('\n')
print(tickers)

['AMZN', 'INTC', 'LRCX', 'META', 'BABA', 'GIS', 'KMI', 'KO', 'MCD', 'MDT', 'MMM', 'TEL', 'USB', 'XYL', 'URNM', 'ABNB', 'AMAT', 'ASML', 'KLAC', 'KLIC', 'MRAM', 'SWKS', 'UROY', 'VMFXX', 'BWXT', 'DOCS', 'SPOT', 'SILXY', 'CLLNY', 'SRUUF', 'YLLXF', 'IGN', 'VGT', 'VUG', 'SOXX', 'VONG', 'JEPI', 'SCHD', 'VB', 'VIG', 'VTI', 'FXAIX', 'SPAXX', 'VXUS', 'TWCGX', 'SPY', 'EMCLOUD', 'COMP', 'FROG', 'MCHP', 'IRDM', 'NVEC', 'MXL', 'OPRA', 'BELFB', 'EXTR', 'IDCC', 'ON', 'HLIT', 'PLAB', 'VEEV', 'TTD', 'SHOP', 'CDLX', 'ZI', 'DV', 'S', 'ZS', 'CRWD', 'ADSK', 'PANW', 'MSI', 'ANET', 'EQIX', 'EGIO', 'DOCN', 'NET', 'RKLB', 'ASTS', 'COIN', 'NCNO', 'FISV', 'ENPH', 'TOELY', 'QRVO', 'BESI', 'VSH', 'AEHR', 'TXN', 'SLAB', 'CRUS', 'NXPI', 'AVGO', 'MPWR', 'LSCC', 'SNPS', 'CDNS', 'APH', 'CGNX', 'PLPC', 'SYM', 'IOT', 'TMO', 'DHR', 'TT', 'IRM', 'SEIC', 'NOW', 'BDC', 'DIOD', 'JBL', 'CARG', 'CSGS', 'ADBE', 'CRM', 'NOW', 'SHOP', 'WDAY', 'PYPL', 'SNOW', 'CRWD', 'TEAM', 'DDOG', 'SQ', 'MDB', 'VEEV', 'ZS', 'HUBS', 'NET', 'ZM', 'O

Full Year Data

In [3]:
# set the API endpoint URL with a placeholder for the ticker
url_template = "https://fmpcloud.io/api/v3/key-metrics/{}?datatype=csv&apikey=a726d5acabd86910ef2749315f5eaa86"

# define a list of tickers
tickers = tickers

# loop over the tickers and make an API request for each one
for ticker in tickers:
    # replace the placeholder with the current ticker
    url = url_template.format(ticker)

    # make an HTTP GET request to the API endpoint and retrieve the data in CSV format
    response = requests.get(url)
    data = response.text

    # create a dataframe from the data
    df = pd.read_csv(io.StringIO(data))

    # add a new column to the dataframe with the ticker symbol
    df['Ticker'] = ticker

    # store the dataframe in the ticker_dataframes dictionary with the ticker as the key
    ticker_dataframes[ticker] = df

# concatenate all dataframes into one
combined_df = pd.concat(ticker_dataframes.values(), ignore_index=True)

# save the combined dataframe as a CSV file
combined_df.to_csv("metricsFY.csv", index=False)


NameError: name 'ticker_dataframes' is not defined

In [ ]:
import pandas as pd

# Read the CSV file using pandas
df = pd.read_csv('metricsFY.csv')

# Define the DW table columns
desired_columns = [
    "symbol",
    "date",
    "period",
    "revenuePerShare",
    "netIncomePerShare",
    "operatingCashFlowPerShare",
    "freeCashFlowPerShare",
    "cashPerShare",
    "bookValuePerShare",
    "tangibleBookValuePerShare",
    "peRatio",
    "priceToSalesRatio",
    "pfcfRatio",
    "pbRatio",
    "evToSales",
    "evToOperatingCashFlow",
    "evToFreeCashFlow",
    "freeCashFlowYield",
    "debtToEquity",
    "debtToAssets",
    "currentRatio",
    "dividendYield",
    "salesGeneralAndAdministrativeToRevenue",
    "researchAndDdevelopementToRevenue",
    "intangiblesToTotalAssets",
    "capexToOperatingCashFlow",
    "capexToRevenue",
    "stockBasedCompensationToRevenue",
    "roic",
    "returnOnTangibleAssets",
    "averageInventory",
    "daysSalesOutstanding",
    "daysOfInventoryOnHand",
    "inventoryTurnover",
    "roe",
    "capexPerShare"
]

# Keep only the desired columns
df = df[desired_columns]

# Save the resulting DataFrame to a new CSV file
df.to_csv('metricsFY.csv', index=False)

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

# Read the CSV file into a DataFrame
key_metrics = pd.read_csv('metricsFY.csv')


# Handle Non-finite Values:
key_metrics['averageInventory'] = key_metrics['averageInventory'].fillna(0).astype(int)
key_metrics['daysSalesOutstanding'] = key_metrics['daysSalesOutstanding'].fillna(0).astype(int)
key_metrics['daysOfInventoryOnHand'] = key_metrics['daysOfInventoryOnHand'].fillna(0).astype(int)


# Explicitly set the 'DATE' column as datetime64[ns]
key_metrics = pd.DataFrame({
    'SYMBOL': key_metrics['symbol'].astype(str),
    'DATE': pd.to_datetime(key_metrics['date'], errors='coerce'),
    'PERIOD': key_metrics['period'].astype(str),
    'REVENUEPERSHARE': key_metrics['revenuePerShare'].astype(float),
    'NETINCOMEPERSHARE': key_metrics['netIncomePerShare'].astype(float),
    'OPERATINGCASHFLOWPERSHARE': key_metrics['operatingCashFlowPerShare'].astype(float),
    'FREECASHFLOWPERSHARE': key_metrics['freeCashFlowPerShare'].astype(float),
    'CASHPERSHARE': key_metrics['cashPerShare'].astype(float),
    'BOOKVALUEPERSHARE': key_metrics['bookValuePerShare'].astype(float),
    'TANGIBLEBOOKVALUEPERSHARE': key_metrics['tangibleBookValuePerShare'].astype(float),
    'PERATIO': key_metrics['peRatio'].astype(float),
    'PRICETOSALESRATIO': key_metrics['priceToSalesRatio'].astype(float),
    'PFCFRATIO': key_metrics['pfcfRatio'].astype(float),
    'PBRATIO': key_metrics['pbRatio'].astype(float),
    'EVTOSALES': key_metrics['evToSales'].astype(float),
    'EVTOOPERATINGCASHFLOW': key_metrics['evToOperatingCashFlow'].astype(float),
    'EVTOFREECASHFLOW': key_metrics['evToFreeCashFlow'].astype(float),
    'FREECASHFLOWYIELD': key_metrics['freeCashFlowYield'].astype(float),
    'DEBTTOEQUITY': key_metrics['debtToEquity'].astype(float),
    'DEBTTOASSETS': key_metrics['debtToAssets'].astype(float),
    'CURRENTRATIO': key_metrics['currentRatio'].astype(float),
    'DIVIDENDYIELD': key_metrics['dividendYield'].astype(float),
    'SALESGENERALANDADMINISTRATIVETOREVENUE': key_metrics['salesGeneralAndAdministrativeToRevenue'].astype(float),
    'RESEARCHANDDEVELOPMENTTOREVENUE': key_metrics['researchAndDdevelopementToRevenue'].astype(float),
    'INTANGIBLESTOTOTALASSETS': key_metrics['intangiblesToTotalAssets'].astype(float),
    'CAPEXTOOPERATINGCASHFLOW': key_metrics['capexToOperatingCashFlow'].astype(float),
    'CAPEXTOREVENUE': key_metrics['capexToRevenue'].astype(float),
    'STOCKBASEDCOMPENSATIONTOREVENUE': key_metrics['stockBasedCompensationToRevenue'].astype(float),
    'ROIC': key_metrics['roic'].astype(float),
    'RETURNONTANGIBLEASSETS': key_metrics['returnOnTangibleAssets'].astype(float),
    'AVERAGEINVENTORY': key_metrics['averageInventory'].astype(int),
    'DAYSSALESOUTSTANDING': key_metrics['daysSalesOutstanding'].astype(int),
    'DAYSOFINVENTORYONHAND': key_metrics['daysOfInventoryOnHand'].astype(int),
    'INVENTORYTURNOVER': key_metrics['inventoryTurnover'].astype(float),
    'ROE': key_metrics['roe'].astype(float),
    'CAPEXPERSHARE': key_metrics['capexPerShare'].astype(float),
})

# Check if the column exists before renaming
old_column_name = 'researchAndDdevelopementToRevenue'
new_column_name = 'researchAndDevelopementToRevenue'

if old_column_name in key_metrics.columns:
    key_metrics.rename(columns={old_column_name: new_column_name}, inplace=True)


key_metrics.columns = key_metrics.columns.str.upper()


project_id = 'stock-market-data-391622'
dataset_id = 'market_information'
table_name = 'key_metrics'


# Create a BigQuery client with service account credentials
credentials = service_account.Credentials.from_service_account_file('stock-market-data-391622-03ee9e592be9.json')
client = bigquery.Client(credentials=credentials, project=project_id)

# Specify the destination table in BigQuery
destination_table = 'stock-market-data-391622.market_information.key_metrics'


# Write the DataFrame to BigQuery without specifying the schema
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # Options: "WRITE_TRUNCATE", "WRITE_APPEND", "WRITE_EMPTY", "WRITE_TRUNCATE"
)


client.load_table_from_dataframe(
    key_metrics, destination_table, job_config=job_config
).result()

In [ ]:
job_id = 'bcfff2f4-59a3-4e04-ae8e-2611f539f067'  # Replace with new job ID
job = client.get_job(job_id)
print(job.state)

Quarterly Data

In [ ]:
# set the API endpoint URL with a placeholder for the ticker
url_template = "https://fmpcloud.io/api/v3/key-metrics/{}?datatype=csv&period=quarter&apikey=a726d5acabd86910ef2749315f5eaa86"

# define a list of tickers
tickers = tickers

# loop over the tickers and make an API request for each one
for ticker in tickers:
    # replace the placeholder with the current ticker
    url = url_template.format(ticker)

    # make an HTTP GET request to the API endpoint and retrieve the data in CSV format
    response = requests.get(url)
    data = response.text

    # create a dataframe from the data
    df = pd.read_csv(io.StringIO(data))

    # add a new column to the dataframe with the ticker symbol
    df['Ticker'] = ticker

    # store the dataframe in the ticker_dataframes dictionary with the ticker as the key
    ticker_dataframes[ticker] = df

# concatenate all dataframes into one
combined_df = pd.concat(ticker_dataframes.values(), ignore_index=True)

# save the combined dataframe as a CSV file
combined_df.to_csv("metrics_quarterly.csv", index=False)


In [ ]:
import pandas as pd

# Read the CSV file using pandas
df = pd.read_csv('metrics_quarterly.csv')

# Define the DW table columns
desired_columns = [
    "symbol",
    "date",
    "period",
    "revenuePerShare",
    "netIncomePerShare",
    "operatingCashFlowPerShare",
    "freeCashFlowPerShare",
    "cashPerShare",
    "bookValuePerShare",
    "tangibleBookValuePerShare",
    "peRatio",
    "priceToSalesRatio",
    "pfcfRatio",
    "pbRatio",
    "evToSales",
    "evToOperatingCashFlow",
    "evToFreeCashFlow",
    "freeCashFlowYield",
    "debtToEquity",
    "debtToAssets",
    "currentRatio",
    "dividendYield",
    "salesGeneralAndAdministrativeToRevenue",
    "researchAndDdevelopementToRevenue",
    "intangiblesToTotalAssets",
    "capexToOperatingCashFlow",
    "capexToRevenue",
    "stockBasedCompensationToRevenue",
    "roic",
    "returnOnTangibleAssets",
    "averageInventory",
    "daysSalesOutstanding",
    "daysOfInventoryOnHand",
    "inventoryTurnover",
    "roe",
    "capexPerShare"
]

# Keep only the desired columns
df = df[desired_columns]

# Save the resulting DataFrame to a new CSV file
df.to_csv('metrics_quarterly.csv', index=False)

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

# Read the CSV file into a DataFrame
key_metrics = pd.read_csv('metrics_quarterly.csv')


# Handle Non-finite Values:
key_metrics['averageInventory'] = key_metrics['averageInventory'].fillna(0).astype(int)
key_metrics['daysSalesOutstanding'] = key_metrics['daysSalesOutstanding'].fillna(0).astype(int)
key_metrics['daysOfInventoryOnHand'] = key_metrics['daysOfInventoryOnHand'].fillna(0).astype(int)


# Explicitly set the 'DATE' column as datetime64[ns]
key_metrics = pd.DataFrame({
    'SYMBOL': key_metrics['symbol'].astype(str),
    'DATE': pd.to_datetime(key_metrics['date'], errors='coerce'),
    'PERIOD': key_metrics['period'].astype(str),
    'REVENUEPERSHARE': key_metrics['revenuePerShare'].astype(float),
    'NETINCOMEPERSHARE': key_metrics['netIncomePerShare'].astype(float),
    'OPERATINGCASHFLOWPERSHARE': key_metrics['operatingCashFlowPerShare'].astype(float),
    'FREECASHFLOWPERSHARE': key_metrics['freeCashFlowPerShare'].astype(float),
    'CASHPERSHARE': key_metrics['cashPerShare'].astype(float),
    'BOOKVALUEPERSHARE': key_metrics['bookValuePerShare'].astype(float),
    'TANGIBLEBOOKVALUEPERSHARE': key_metrics['tangibleBookValuePerShare'].astype(float),
    'PERATIO': key_metrics['peRatio'].astype(float),
    'PRICETOSALESRATIO': key_metrics['priceToSalesRatio'].astype(float),
    'PFCFRATIO': key_metrics['pfcfRatio'].astype(float),
    'PBRATIO': key_metrics['pbRatio'].astype(float),
    'EVTOSALES': key_metrics['evToSales'].astype(float),
    'EVTOOPERATINGCASHFLOW': key_metrics['evToOperatingCashFlow'].astype(float),
    'EVTOFREECASHFLOW': key_metrics['evToFreeCashFlow'].astype(float),
    'FREECASHFLOWYIELD': key_metrics['freeCashFlowYield'].astype(float),
    'DEBTTOEQUITY': key_metrics['debtToEquity'].astype(float),
    'DEBTTOASSETS': key_metrics['debtToAssets'].astype(float),
    'CURRENTRATIO': key_metrics['currentRatio'].astype(float),
    'DIVIDENDYIELD': key_metrics['dividendYield'].astype(float),
    'SALESGENERALANDADMINISTRATIVETOREVENUE': key_metrics['salesGeneralAndAdministrativeToRevenue'].astype(float),
    'RESEARCHANDDEVELOPMENTTOREVENUE': key_metrics['researchAndDdevelopementToRevenue'].astype(float),
    'INTANGIBLESTOTOTALASSETS': key_metrics['intangiblesToTotalAssets'].astype(float),
    'CAPEXTOOPERATINGCASHFLOW': key_metrics['capexToOperatingCashFlow'].astype(float),
    'CAPEXTOREVENUE': key_metrics['capexToRevenue'].astype(float),
    'STOCKBASEDCOMPENSATIONTOREVENUE': key_metrics['stockBasedCompensationToRevenue'].astype(float),
    'ROIC': key_metrics['roic'].astype(float),
    'RETURNONTANGIBLEASSETS': key_metrics['returnOnTangibleAssets'].astype(float),
    'AVERAGEINVENTORY': key_metrics['averageInventory'].astype(int),
    'DAYSSALESOUTSTANDING': key_metrics['daysSalesOutstanding'].astype(int),
    'DAYSOFINVENTORYONHAND': key_metrics['daysOfInventoryOnHand'].astype(int),
    'INVENTORYTURNOVER': key_metrics['inventoryTurnover'].astype(float),
    'ROE': key_metrics['roe'].astype(float),
    'CAPEXPERSHARE': key_metrics['capexPerShare'].astype(float),
})

# Check if the column exists before renaming
old_column_name = 'researchAndDdevelopementToRevenue'
new_column_name = 'researchAndDevelopementToRevenue'

if old_column_name in key_metrics.columns:
    key_metrics.rename(columns={old_column_name: new_column_name}, inplace=True)


key_metrics.columns = key_metrics.columns.str.upper()


project_id = 'stock-market-data-391622'
dataset_id = 'market_information'
table_name = 'key_metrics'


# Create a BigQuery client with service account credentials
credentials = service_account.Credentials.from_service_account_file('stock-market-data-391622-03ee9e592be9.json')
client = bigquery.Client(credentials=credentials, project=project_id)

# Specify the destination table in BigQuery
destination_table = 'stock-market-data-391622.market_information.key_metrics'


# Write the DataFrame to BigQuery without specifying the schema
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # Options: "WRITE_TRUNCATE", "WRITE_APPEND", "WRITE_EMPTY", "WRITE_TRUNCATE"
)


client.load_table_from_dataframe(
    key_metrics, destination_table, job_config=job_config
).result()

In [ ]:
job_id = '75193c05-ec54-4697-9925-e9dd8c731de8'  # Replace with new job ID
job = client.get_job(job_id)
print(job.state)